In [1]:
import pyspark
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

import os

In [2]:
# Create Spark Session
spark = SparkSession.builder.appName("Training").getOrCreate()

# Define data path
BASE_PATH = os.getcwd()
DATA_PATH = os.path.join(BASE_PATH, 'data')
TRAINING_DATASET_PATH = os.path.join(DATA_PATH, 'training')
MODEL_PATH = os.path.join(BASE_PATH, 'model')

In [3]:
def get_pipeline(data: pyspark.sql.DataFrame):
    # Creates an ML Pipeline and appropriate Param Map for Cross Validation 
    
    
    #Index labels, adding metadata to the label column.
    # Fit on whole dataset to include all labels in index.
    labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

    # Automatically identify categorical features, and index them.
    # Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer =\
        VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

    # Train a RandomForest model.
    rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

    # Convert indexed labels back to original labels.
    labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                                   labels=labelIndexer.labels)
    
    paramMap = ParamGridBuilder() \
        .addGrid(rf.numTrees, [5, 10, 15, 20]) \
        .build()
    
    return (Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter]), paramMap)

In [4]:
def get_model(pipeline: pyspark.ml.Pipeline, 
              paramGrid, evaluator, 
              n_folds: int, 
              dataset: pyspark.sql.DataFrame, 
              model_path):
    # Creates a cross validation evaluator according to defined pipeline and param maps.
    # Finds the best model according to the cross validation results
    # Persists the model to the given model path
    model = CrossValidator(estimator=pipeline,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           numFolds=n_folds)
    cv_model = model.fit(dataset)
    model = cv_model.bestModel
    evaluation = list(zip(cv_model.avgMetrics, paramGrid))
    model.write().overwrite().save(model_path)
    return model, evaluation

In [5]:
def training(training_data_path: str, model_save_path: str):
    # Given training data path and model path
    # Produces and persists an ML Model based on training data and defined configuration
    
    # Load training dataset
    training_df = spark.read.parquet(TRAINING_DATASET_PATH)
    
    # Get pipeline configuration
    pipeline, paramMap = get_pipeline(training_df)
    evaluator = MulticlassClassificationEvaluator(
        labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
    num_folds = 2
    
    # Create and persist ML Model based on training data and pipeline configuration
    model, evaluations = get_model(pipeline=pipeline,
                                   paramGrid=paramMap,
                                   evaluator=evaluator,
                                   n_folds=num_folds,
                                   dataset=training_df,
                                   model_path=os.path.join(BASE_PATH, 'model'))
    return model, evaluations

In [7]:
model, evaluations = training(training_data_path=TRAINING_DATASET_PATH,
                              model_save_path=MODEL_PATH)

rf = model.stages[2]
print(f'Best Model: \n'\
      f'Random Forest model with {rf.getNumTrees} trees.\n'\
      f'Saved to {MODEL_PATH}\n\n'\
      'Cross Validated models and average results:')

for evaluation, params in evaluations:
    print(f'Model: Random Forest model with {list(params.values())[0]} trees\t\tEvaluation: {evaluation}')


Best Model: 
Random Forest model with 15 trees.
Saved to /Users/orz/projects/iguazio/model_example/model

Cross Validated models and average results:
Model: Random Forest model with 5 trees		Evaluation: 0.9777777777777779
Model: Random Forest model with 10 trees		Evaluation: 0.9777777777777779
Model: Random Forest model with 15 trees		Evaluation: 0.9888888888888889
Model: Random Forest model with 20 trees		Evaluation: 0.9888888888888889
